# Create the full flow here : Only using Python functions no intelligent algorithms

In [1]:
!pip install llama-agents llama-index-agent-openai pandas

  Using cached llama_agents-0.0.12-py3-none-any.whl.metadata (11 kB)
  Using cached fastapi-0.109.2-py3-none-any.whl.metadata (25 kB)
  Using cached pytest_asyncio-0.23.8-py3-none-any.whl.metadata (3.9 kB)
  Using cached pytest_mock-3.14.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached textual-0.70.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached starlette-0.36.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached pytest-8.3.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached linkify_it_py-2.0.3-py3-none-any.whl.metadata (8.5 kB)
  Using cached mdit_py_plugins-0.4.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached iniconfig-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached tomli-2.0.1-py3-none-any.whl.metadata (8.9 kB)
  U

In [4]:
import pandas as pd

data_path = "/Users/deveshsurve/UNIVERSITY/INFO/7374/Final-Project-SmartCommerce/data_pipeline/data_files/"
# Load datasets
customers = pd.read_csv( data_path + 'olist_customers_dataset.csv')
# geolocations = pd.read_csv('olist_geolocation_dataset.csv')
order_items = pd.read_csv(data_path +'olist_order_items_dataset.csv')
payments = pd.read_csv(data_path +'olist_order_payments_dataset.csv')
reviews = pd.read_csv(data_path +'olist_order_reviews_dataset.csv')
orders = pd.read_csv(data_path +'olist_orders_dataset.csv')
products = pd.read_csv(data_path +'olist_products_dataset.csv')
sellers = pd.read_csv(data_path +'olist_sellers_dataset.csv')
product_category_translation = pd.read_csv(data_path +'product_category_name_translation.csv')

In [6]:
def preprocess_data(customers, orders, order_items, order_reviews, products):
    # Merge dataframes as needed
    data = orders.merge(customers, on='customer_id', how='left')
    data = data.merge(order_items, on='order_id', how='left')
    data = data.merge(order_reviews, on='order_id', how='left')
    data = data.merge(products, on='product_id', how='left')
    return data

def get_preprocessed_data() -> pd.DataFrame:
    data = preprocess_data(customers, orders, order_items, reviews, products)
    return data

from llama_index.core.tools import FunctionTool

preprocess_tool = FunctionTool.from_defaults(fn=get_preprocessed_data)


In [7]:
def generate_recommendations(data, customer_id):
    # For simplicity, recommend top 5 products based on some criteria
    customer_data = data[data['customer_id'] == customer_id]
    recommended_products = customer_data['product_id'].value_counts().head(5).index.tolist()
    return recommended_products

def get_recommendations(customer_id: str) -> list:
    data = get_preprocessed_data()
    recommendations = generate_recommendations(data, customer_id)
    return recommendations

recommend_tool = FunctionTool.from_defaults(fn=get_recommendations)


In [8]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_agents import AgentService, ControlPlaneServer, SimpleMessageQueue, LocalLauncher, AgentOrchestrator

# Define data processing agent
preprocess_worker = FunctionCallingAgentWorker.from_tools([preprocess_tool], llm=OpenAI())
preprocess_agent = preprocess_worker.as_agent()

preprocess_service = AgentService(
    agent=preprocess_agent,
    message_queue=SimpleMessageQueue(),
    description="Data processing service",
    service_name="data_processor"
)

# Define recommendation agent
recommend_worker = FunctionCallingAgentWorker.from_tools([recommend_tool], llm=OpenAI())
recommend_agent = recommend_worker.as_agent()

recommend_service = AgentService(
    agent=recommend_agent,
    message_queue=SimpleMessageQueue(),
    description="Recommendation service",
    service_name="recommender"
)


In [9]:
import dotenv
import logging

# Load environment variables
dotenv.load_dotenv()

# Set up logging
logging.getLogger("llama_agents").setLevel(logging.INFO)

# Set up message queue and control plane
message_queue = SimpleMessageQueue()
control_plane = ControlPlaneServer(
    message_queue=message_queue,
    orchestrator=AgentOrchestrator(llm=OpenAI())
)

# Launch the services
launcher = LocalLauncher(
    [preprocess_service, recommend_service],
    control_plane,
    message_queue,
)

# Run a sample query
customer_id = '06b8999e2fba1a1fbc88172c00ba8bc7'  # Example customer ID
result = launcher.launch_single(f"Recommend products for customer {customer_id}")
print(result)


RuntimeError: asyncio.run() cannot be called from a running event loop